# Step 9: Improving Deployment Efficiency with ONNX

## 9.1 The Challenge: From Research to Production

After successfully developing, training, and evaluating our temperature forecasting model, the final step is to transition it from a research environment (like this notebook) to a live, production application. This process, known as **model deployment**, presents several critical challenges:

*   **Dependency and Integration Complexity:** Our model was trained using the CatBoost library in a Python environment. Deploying it requires replicating these dependencies in a production server, which can be complex and might conflict with existing technology stacks (e.g., a web server running on Java or C#).
*   **Performance and Scalability:** Production systems demand fast inference speed (low latency) and the ability to handle many simultaneous requests (high throughput). Standard model formats are often not optimized for these requirements.
*   **Hardware and Platform Diversity:** A model may need to run on various systems, from high-performance cloud GPUs to resource-limited edge devices. The original model format is not inherently optimized for such diverse hardware.

To address these challenges, we will use **ONNX (Open Neural Network Exchange)**. ONNX is an open-source format that provides a standardized representation of machine learning models. By converting our model to ONNX, we can achieve a portable, high-performance asset ready for robust deployment.

## 9.2 Environment Setup and Imports

First, let's set up the environment by importing the necessary libraries.

In [ ]:
# %pip install onnx onnxruntime scikit-learn==1.2.2 catboost joblib numpy pandas

In [2]:
import os
import joblib
import numpy as np
import pandas as pd
import jinja2

import onnx
import onnxruntime as rt

import timeit  # To benchmark performance
import warnings
warnings.filterwarnings("ignore")

## 9.3 ONNX Application to Hanoi Temperature Forecasting Project

In this section, we apply the ONNX conversion process to both our **Daily** and **Hourly** forecasting models. This will allow us to compare the performance gains across models of different complexities.

### 9.3.1 Loading All Pre-trained Models and Test Data

We begin by loading our two champion models (CatBoost trained on daily data and CatBoost trained on hourly data) and their corresponding preprocessed test datasets.

In [4]:
# --- Define paths for ALL assets ---

# Hourly Model and Data
hourly_model_path = "./Best-Hourly-Model-Hyperparams.joblib"
hourly_data_path = "./Hourly Dataframe Preprocessed.pkl"

# Daily Model and Data
daily_model_path = "./best_daily_model.joblib"
daily_data_path = "./Daily Dataframe Preprocessed.pkl"


# --- Dictionary to hold loaded assets for easier access ---
assets = {}

# --- Load all assets ---
try:
    # Load Hourly assets
    assets['hourly_model'] = joblib.load(hourly_model_path)
    with open(hourly_data_path, 'rb') as f:
        all_hourly_data = joblib.load(f)
    assets['X_test_hourly'] = all_hourly_data['X_test']
    print(f"Successfully loaded Hourly model and data. Shape: {assets['X_test_hourly'].shape}")

    # Load Daily assets
    assets['daily_model'] = joblib.load(daily_model_path)
    with open(daily_data_path, 'rb') as f:
        all_daily_data = joblib.load(f)
    assets['X_test_daily'] = all_daily_data['X_test']
    print(f"Successfully loaded Daily model and data. Shape: {assets['X_test_daily'].shape}")

except FileNotFoundError as e:
    print(f"Error: A file was not found. Please check your file paths and names.")
    print(e)
except Exception as e:
    print(f"An error occurred: {e}")

if 'X_test_hourly' in assets:
    display(assets['X_test_hourly'].head())

Successfully loaded Hourly model and data. Shape: (514, 114)
Successfully loaded Daily model and data. Shape: (549, 93)


,temp,precipprob,windgust,winddir,visibility,day_sin,month,day_of_week,season_Fall,temp_trend_yearly,...,humidity_rolling_last_12h,precip_rolling_last_3h,precip_rolling_last_9h,precip_rolling_last_15h,precipprob_rolling_last_9h,precipprob_rolling_last_15h,windspeed_rolling_last_6h,uvindex_rolling_last_3h,uvindex_rolling_last_6h,uvindex_rolling_last_18h
datetime,,,,,,,,,,,,,,,,,,,,,
2024-04-27,32.195833,0.000000,17.345833,116.962500,7.791667,0.895839,4,5,0,25.281111,...,53.160833,0.0,0.000000,0.000000,0.000000,0.000000,8.350000,0.0,0.166667,4.222222
2024-04-28,30.658333,0.000000,16.850000,97.916667,10.341667,0.888057,4,6,0,25.280007,...,77.934167,0.0,0.000000,0.000000,0.000000,0.000000,13.250000,0.0,0.166667,4.055556
2024-04-29,30.216667,8.333333,20.050000,119.958333,9.695833,0.880012,4,0,0,25.278886,...,80.111667,0.0,0.022222,0.013333,22.222222,13.333333,12.633333,0.0,0.166667,4.000000
2024-04-30,31.137500,4.166667,23.495833,93.158333,9.541667,0.871706,4,1,0,25.277749,...,70.264167,0.0,0.088889,0.053333,11.111111,6.666667,12.166667,0.0,0.166667,4.055556
2024-05-01,26.041667,41.666667,21.641667,76.458333,9.500000,0.863142,5,2,0,25.276594,...,84.022500,0.1,0.033333,0.046667,33.333333,40.000000,9.933333,0.0,0.000000,0.888889


### 9.3.2 Converting Both Models to ONNX Format

We will now convert both the Daily and Hourly models into their respective `.onnx` files. The process is identical for both, demonstrating the consistency of the `save_model` method.



In [5]:
import os

# --- Define the output directory ---
ONNX_OUTPUT_DIR = "output_onnx_models"
os.makedirs(ONNX_OUTPUT_DIR, exist_ok=True)

# --- Define a dictionary to hold the paths for ALL ONNX models ---
onnx_paths = {
    'daily': [],
    'hourly': []
}

# --- Loop through and convert all base models for both Daily and Hourly pipelines ---
for model_type in ['daily', 'hourly']:
    print(f"--- Converting All 5 Base Models for {model_type.title()} Pipeline ---")
    try:
        wrapper_model = assets[f'{model_type}_model']
        X_test_df = assets[f'X_test_{model_type}']

        # Ensure column names are strings once
        if not all(isinstance(col, str) for col in X_test_df.columns):
            X_test_df.columns = [str(col) for col in X_test_df.columns]

        # Loop through each of the 5 base estimators
        for i, base_model in enumerate(wrapper_model.estimators_):
            # Create a file path INSIDE the defined directory
            output_path = f"{ONNX_OUTPUT_DIR}/hanoi_{model_type}_t_plus_{i+1}.onnx"

            # Add the file path (which includes the directory) to the dict
            onnx_paths[model_type].append(output_path)

            # Perform conversion
            base_model.save_model(output_path, format="onnx")

            # (The os.path.exists check is good, keep it)
            if os.path.exists(output_path):
                print(f"  Converted model for t+{i+1}. Saved to '{output_path}'")
            else:
                print(f"  Failed to convert model for t+{i+1}.")
        print("") # Add a newline after processing one model type

    except Exception as e:
        print(f"An error occurred during {model_type} model conversion: {e}\n")

--- Converting All 5 Base Models for Daily Pipeline ---
  Converted model for t+1. Saved to 'output_onnx_models/hanoi_daily_t_plus_1.onnx'
  Converted model for t+2. Saved to 'output_onnx_models/hanoi_daily_t_plus_2.onnx'
  Converted model for t+3. Saved to 'output_onnx_models/hanoi_daily_t_plus_3.onnx'
  Converted model for t+4. Saved to 'output_onnx_models/hanoi_daily_t_plus_4.onnx'
  Converted model for t+5. Saved to 'output_onnx_models/hanoi_daily_t_plus_5.onnx'

--- Converting All 5 Base Models for Hourly Pipeline ---
  Converted model for t+1. Saved to 'output_onnx_models/hanoi_hourly_t_plus_1.onnx'
  Converted model for t+2. Saved to 'output_onnx_models/hanoi_hourly_t_plus_2.onnx'
  Converted model for t+3. Saved to 'output_onnx_models/hanoi_hourly_t_plus_3.onnx'
  Converted model for t+4. Saved to 'output_onnx_models/hanoi_hourly_t_plus_4.onnx'
  Converted model for t+5. Saved to 'output_onnx_models/hanoi_hourly_t_plus_5.onnx'



## 9.3.3 Defining the Benchmark and Verification Functions

The next code cells will define the two core utility functions needed to run our final benchmark. These functions are:

1.  **`run_full_pipeline_benchmark(...)`**: This is the main function, responsible for handling all the logic for testing speed and correctness.
2.  **`compare_model_sizes(...)`**: This is a helper function focused on comparing file storage.

### I. **`run_full_pipeline_benchmark()`**

* **Load Assets:** Loads the original `.joblib` model (which is a `MultiOutputRegressor`) and the 5 corresponding `.onnx` files into inference sessions.
* **Verify Correctness:** Performs a "5-vs-5" check. It runs a full 5-horizon prediction with `.joblib` and compares it against the combined result of all 5 `.onnx` sessions to ensure they are numerically identical.
* **Run Performance Benchmark:** Runs a fair speed test, timing the *full 5-day forecast* for `model.predict()` (Joblib) against the time it takes to run *all 5 `session.run()` calls* (ONNX).
* **Store Results:** Appends the final metrics (Avg. Time per forecast, Throughput) to a results list, ready for summary.

### II. **`compare_model_sizes()`**

* **Measure Joblib:** Gets the total size (MB) of the 2 original `.joblib` files.
* **Measure ONNX:** Gets the total size (MB) of all 10 exported `.onnx` files from the `output_onnx_models` folder.
* **Print Conclusion:** Compares the two totals and prints a final conclusion (e.g., `ONNX IS 3.4x LARGER THAN JOBLIB`).

In [7]:
import onnxruntime as rt
import timeit
import numpy as np

# Speed comparison function
def run_full_pipeline_benchmark(model_type, assets, onnx_paths_dict, results_list, num_runs=10):
    """
    Run a full 5-vs-5 benchmark for a model type ('daily' or 'hourly').

    This function will:
    1. Load the .joblib model and the 5 corresponding .onnx sessions.
    2. Verify correctness (5-vs-5).
    3. Run the performance benchmark (5-vs-5).
    4. Append the results to `results_list`.

    Args:
        model_type (str): 'daily' or 'hourly'.
        assets (dict): Dict containing the .joblib models and dataframes (e.g., assets['daily_model']).
        onnx_paths_dict (dict): Dict containing a LIST of .onnx paths (e.g., onnx_paths_dict['daily']).
        results_list (list): The list to append benchmark results to.
        num_runs (int): The number of benchmark runs for timeit.
    """
    try:
        # RETAIN: Start processing
        print(f"--- Processing {model_type.title()} Pipeline ---")

        # --- 1. SETUP ---
        original_model = assets[f'{model_type}_model']

        # Load data and convert to numpy float32 inside the function
        X_test_df = assets[f'X_test_{model_type}']
        X_test_np = X_test_df.astype(np.float32).values

        onnx_sessions = []
        model_onnx_paths = onnx_paths_dict[model_type] # Get the list of paths

        for path in model_onnx_paths:
            onnx_sessions.append(rt.InferenceSession(path))

        # Get input name (assuming all 5 models have the same)
        input_name = onnx_sessions[0].get_inputs()[0].name

        # --- 2. CORRECTNESS VERIFICATION ---
        original_preds = original_model.predict(X_test_np)
        onnx_preds = []

        for sess in onnx_sessions:
            result = sess.run(None, {input_name: X_test_np})[0]
            onnx_preds.append(result.flatten())

        # Transpose ONNX preds to match shape (samples, 5)
        onnx_preds_array = np.array(onnx_preds).T

        np.testing.assert_allclose(original_preds, onnx_preds_array, rtol=1e-5)
        # RETAIN: Successful verification is important
        print("Correctness Verified: Predictions match.")

        # --- 3. PERFORMANCE BENCHMARK (ACCURATE 5-vs-5) ---

        # Time the full .joblib pipeline
        t_original_5_models = timeit.timeit(lambda: original_model.predict(X_test_np), number=num_runs)

        # Define a function that runs all 5 ONNX sessions
        def predict_5_days_onnx_accurate():
            # Use list comprehension for performance
            predictions = [sess.run(None, {input_name: X_test_np}) for sess in onnx_sessions]
            return predictions

        # Time the full ONNX pipeline
        t_onnx_5_models = timeit.timeit(lambda: predict_5_days_onnx_accurate(), number=num_runs)

        # --- 4. STORE RESULTS ---
        num_forecasts = len(X_test_np)
        results_list.append({
            'Model': f"CatBoost ({model_type.title()})", 'Deployment Type': '.joblib (Full 5-Day)',
            'Avg. Time (µs/forecast)': (t_original_5_models / num_runs / num_forecasts) * 1_000_000,
            'Throughput (forecasts/sec)': num_forecasts / (t_original_5_models / num_runs)
        })
        results_list.append({
            'Model': f"CatBoost ({model_type.title()})", 'Deployment Type': 'ONNX (Full 5-Day)',
            'Avg. Time (µs/forecast)': (t_onnx_5_models / num_runs / num_forecasts) * 1_000_000,
            'Throughput (forecasts/sec)': num_forecasts / (t_onnx_5_models / num_runs)
        })

        print(f"Benchmark Complete.\n")
        return True

    except AssertionError:
        print(f"Correctness Check Failed for {model_type}: Predictions do not match! Skipping benchmark.\n")
        return False
    except Exception as e:
        print(f"An error occurred while processing {model_type}: {e}\n")
        return False

# Size comparison function
def compare_model_sizes(
    joblib_daily: str = "best_daily_model.joblib",
    joblib_hourly: str = "Best-Hourly-Model-Hyperparams.joblib",
    onnx_folder: str = "output_onnx_models"
):
    import os

    print("=== MODEL SIZE COMPARISON (Joblib vs ONNX) ===\n")

    # Joblib sizes
    daily_size = os.path.getsize(joblib_daily) / (1024**2) if os.path.exists(joblib_daily) else None
    hourly_size = os.path.getsize(joblib_hourly) / (1024**2) if os.path.exists(joblib_hourly) else None

    print(f"{'Original Joblib':<35} Size")
    print("-" * 55)
    print(f"best_daily_model.joblib               → {daily_size:.2f} MB" if daily_size else "best_daily_model.joblib               → File not found")
    print(f"Best-Hourly-Model-Hyperparams.joblib→ {hourly_size:.2f} MB" if hourly_size else "Best-Hourly-Model-Hyperparams.joblib→ File not found")
    total_joblib = (daily_size or 0) + (hourly_size or 0)
    print(f"{'TOTAL 2 JOBLIB FILES':<35} → {total_joblib:.2f} MB\n")

    # ONNX sizes
    if not os.path.exists(onnx_folder):
        print(f"ONNX folder '{onnx_folder}' does not exist!")
        return

    onnx_files = []
    onnx_total = 0
    for f in sorted(os.listdir(onnx_folder)):
        if f.endswith(".onnx"):
            path = os.path.join(onnx_folder, f)
            size_mb = os.path.getsize(path) / (1024**2)
            onnx_total += size_mb
            onnx_files.append((f, size_mb))

    print(f"{'ONNX files':<35} Size")
    print("-" * 55)
    for name, size in onnx_files:
        print(f"{name:<35} → {size:.2f} MB")
    print(f"{'TOTAL 10 ONNX FILES':<35} → {onnx_total:.2f} MB")

    # Conclusion
    if daily_size and hourly_size and len(onnx_files) == 10:
        print(f"\n{'='*60}")

        ratio = onnx_total / total_joblib

        if ratio > 1.0:
            print(f"CONCLUSION: ONNX IS {ratio:.1f}x LARGER THAN JOBLIB")
            print(f"   (Total ONNX: {onnx_total:.1f} MB vs Joblib: {total_joblib:.1f} MB)")

        elif ratio < 1.0:
            # Calculate inverse ratio for easier reading (e.g., 2.5x smaller)
            reverse_ratio = 1 / ratio
            print(f"CONCLUSION: ONNX IS {reverse_ratio:.1f}x SMALLER THAN JOBLIB")

        else:
            # Rare case: they are equal
            print(f"CONCLUSION: ONNX AND JOBLIB ARE THE SAME SIZE")
            print(f"   (Total ONNX: {onnx_total:.1f} MB vs Joblib: {total_joblib:.1f} MB)")

        print(f"{'='*60}")
    else:
        print("\nNot enough files to compare accurately (need 2 joblib + 10 onnx)")

In [8]:
# --- 5. SUMMARIZE AND DISPLAY RESULTS  ---
# Empty list to store results
benchmark_results = []

# Run for 'daily' and 'hourly' using the created benchmark function
run_full_pipeline_benchmark('daily', assets, onnx_paths, benchmark_results)
run_full_pipeline_benchmark('hourly', assets, onnx_paths, benchmark_results)

# --- 2. Summarize and Display Speed Results ---
if not benchmark_results:
    print("No benchmark results to display. There might have been an error in the previous step.")
else:
    # Convert results to a pandas DataFrame
    df_results = pd.DataFrame(benchmark_results)

    # Use set_index to create a more intuitive, multi-level grouped table
    df_results = df_results.set_index(['Model', 'Deployment Type'])

    # Rename columns for clarity in the final table
    df_results.rename(columns={
        'Avg. Time (µs/forecast)': 'Avg. Time per 5-Day Forecast (µs)',
        'Throughput (forecasts/sec)': 'Throughput (Forecasts/sec)'
    }, inplace=True)

    # Calculate speedup factor for each model type
    for model_name in df_results.index.get_level_values(0).unique():
        try:
            # Access the correct rows using the new index names
            baseline_time = df_results.loc[(model_name, '.joblib (Full 5-Day)'), 'Avg. Time per 5-Day Forecast (µs)']
            onnx_time = df_results.loc[(model_name, 'ONNX (Full 5-Day)'), 'Avg. Time per 5-Day Forecast (µs)']

            if onnx_time == 0:
                speedup_factor = float('inf')
            else:
                speedup_factor = baseline_time / onnx_time

            # Add a new 'Speedup' column
            df_results.loc[(model_name, '.joblib (Full 5-Day)'), 'Speedup'] = '1.00x' # Baseline
            df_results.loc[(model_name, 'ONNX (Full 5-Day)'), 'Speedup'] = f'{speedup_factor:.2f}x'

        except KeyError:
            print(f"Warning: Could not calculate speedup for {model_name}. Check Deployment Types.")
            # Assign 'N/A' to all rows for this model_name
            df_results.loc[(model_name, slice(None)), 'Speedup'] = 'N/A'
        except Exception as e:
            print(f"An error occurred calculating speedup for {model_name}: {e}")
            df_results.loc[(model_name, slice(None)), 'Speedup'] = 'Error'


    # --- Display the final styled table ---
    styled_table = df_results.style.format({
        'Avg. Time per 5-Day Forecast (µs)': '{:,.2f}'.format,
        'Throughput (Forecasts/sec)': '{:,.0f}'.format
    }).set_properties(**{'text-align': 'right'}).set_table_styles([
        {'selector': 'th', 'props': [('text-align', 'center')]},
        {'selector': 'th.row_heading', 'props': [('text-align', 'left')]}
    ])

    print("--- Deployment Performance Comparison (Full 5-Day Forecast) ---")
    display(styled_table)

# --- 3. Add spacing ---
print()
print()
print()

# size comparison
# display size comparison
compare_model_sizes(
    joblib_daily= "best_daily_model.joblib",
    joblib_hourly= "Best-Hourly-Model-Hyperparams.joblib",
    onnx_folder= "output_onnx_models"
)

--- Processing Daily Pipeline ---
Correctness Verified: Predictions match.
Benchmark Complete.

--- Processing Hourly Pipeline ---
Correctness Verified: Predictions match.
Benchmark Complete.

--- Deployment Performance Comparison (Full 5-Day Forecast) ---





=== MODEL SIZE COMPARISON (Joblib vs ONNX) ===

Original Joblib                     Size
-------------------------------------------------------
best_daily_model.joblib               → 0.89 MB
Best-Hourly-Model-Hyperparams.joblib→ 1.20 MB
TOTAL 2 JOBLIB FILES                → 2.09 MB

ONNX files                          Size
-------------------------------------------------------
hanoi_daily_t_plus_1.onnx           → 0.60 MB
hanoi_daily_t_plus_2.onnx           → 0.60 MB
hanoi_daily_t_plus_3.onnx           → 0.60 MB
hanoi_daily_t_plus_4.onnx           → 0.60 MB
hanoi_daily_t_plus_5.onnx           → 0.60 MB
hanoi_hourly_t_plus_1.onnx          → 0.82 MB
hanoi_hourly_t_plus_2.onnx          → 0.82 MB
hanoi_hourly_t_plus_3.onnx          → 0.82 MB
hanoi_hourly_t_plus_4.onnx          → 0.82 MB
hanoi_hourly_t_plus_5.onnx          → 0.82 MB
TOTAL 10 ONNX FILES                 → 7.11 MB

CONCLUSION: ONNX IS 3.4x LARGER THAN JOBLIB
   (Total ONNX: 7.1 MB vs Joblib: 2.1 MB)


## **9.4 Deployment Trade-off Analysis: CatBoost vs. ONNX Runtime**

This section summarizes the performance and size comparison between the original CatBoost model (runtime managed by Python/CatBoost native library) and the ONNX-converted model run by the **ONNX Runtime (ORT)**. This analysis uses the results from the **benchmark** to quantify the true trade-offs and make an informed deployment decision.

### **A. Quantifiable Benchmark Results**

The benchmark, measuring the average latency for a *full 5-day forecast*, reveals that the native CatBoost engine is significantly more efficient for this workload.

| Model Type | Deployment | Avg. Time (µs) | Throughput (Forecasts/sec) | Speedup |
| :--- | :--- | ---: | ---: | :--- |
| **CatBoost (Daily)** | `.joblib` (Baseline) | **21.70 µs** | **46,090** | **1.00x** |
| | `ONNX` (Full 5-Day) | 59.16 µs | 16,902 | **0.37x (Slower)** |
| **CatBoost (Hourly)**| `.joblib` (Baseline) | **24.32 µs** | **41,123** | **1.00x** |
| | `ONNX` (Full 5-Day) | 74.91 µs | 13,350 | **0.32x (Slower)** |
| **Size (Total)** | **.joblib** | **2.09 MB** | N/A | **1.00x** |
| | **ONNX** | **7.11 MB** | N/A | **3.4x (Larger)** |

*Note: The native CatBoost C++ engine is **3.1x faster** (1 / 0.32) on CPU, and the compressed `.joblib` files are **3.4x smaller**.*

### **B. Strategic Conclusions**

1.  **Performance vs. Portability:** The benchmark confirms `.joblib` is **3.1x faster** and **3.4x smaller**. The decision to use ONNX is **not** for optimization, but a trade-off: we sacrifice native speed and size for **interoperability**.

2.  **Primary Goal (Interoperability):** The **sole driver** for conversion is to decouple from Python dependencies (CatBoost) and run on the lightweight **ONNX Runtime**, enabling deployment in non-Python environments (C#, Java).

3.  **File Structure:** The **10 `.onnx` files** are correct. They are the result of decomposing the 2 original `MultiOutputRegressor` wrappers (2 models * 5 horizons = 10 files).

**Conclusion:** ONNX is verified **not as a performance optimization**, but as the **essential path for interoperability**, trading speed and size for deployment flexibility.

## **9.5 Conclusion: `.joblib` or `.onnx` for Streamlit**

Based on conclusive benchmark results, the final decision is to **use the original `.joblib` models** for the production Streamlit app. Since Streamlit is a Python-native environment, the native models are the superior choice.

### **1. The Rationale: A Clear Performance Win**

The benchmark proves the native `.joblib` models are significantly more performant in our Python environment:

* **⚡️ Superior Speed:** **Up to 3.1x faster**. The native CatBoost C++ engine is highly specialized and faster (e.g., **24.32 µs**) than the full, general-purpose ONNX pipeline (**74.91 µs**).
* **💾 Superior Size:** **3.4x smaller**. The `.joblib` files (2.09 MB total) use efficient **compression**, while the exported `.onnx` files (7.11 MB total) are uncompressed, prioritizing compatibility.

### **2. The Context: Python (Streamlit) vs. Non-Python**

The primary purpose of ONNX is **interoperability** (e.g., running in C# or Java). Since our Streamlit application is **100% Python**, we do not need this.

Using ONNX in our app would mean knowingly accepting a **~3x performance hit** and a **3.4x increase in memory footprint** for zero practical benefit.

**Conclusion:**

The **`.joblib`** models are the clear and logical choice, guaranteeing the fastest latency and lowest resource usage for our Streamlit users. The generated `.onnx` files will be archived as a valuable asset for any future, **non-Python** use cases.